<div  >
<img src="https://raw.githubusercontent.com/threeML/astromodels/master/docs/media/transp_logo.png" alt="drawing" width="300" align="right"/>
 


<div  >
<img src="https://raw.githubusercontent.com/threeML/threeML/master/logo/logo_sq.png" alt="drawing" width="300" align="right"/>



# X-ray Analysis with 3ML
    
While 3ML can handle a lot of different data/likelihood types a lot of attention was spent on making sure that users familiar with past community standards are able to easily adapt to the 3ML workflow. There are some guides for these users in the [documentation](https://threeml.readthedocs.io/en/stable/xspec_users.html).
    
X-ray analysis in 3ML is centered around the `OGIPLike` plugin which reads OGIP style PHAI/II, RMF, and ARF files. the OGIPLike plugin is a specialized version of the `DispersionSpectrumLike` plugin which deals with count data that are produced by convolving the model spectrum with the resonse of an instrument that suffers from energy dispersion. Thus, if you have an instrument you are designing and you don't like fits files... inherit from DispersionSpectrumLike and create your own unique plugin for ROOT, HDF5, txt, etc. files. The cool thing is that you can still fit your data along with normal OGIP type data... or any of the other plugins in the 3ML family. 3ML is a toolbox to bring instruments (and people) together. 
    
    
Let's explore the OGIPLike plugin

 


## The OGIPLike plugin

The OGIP plugin reads in standard OGIP files. **It will complain a lot if files are in the correct format!**. For PHAII files with multiple spectra, you can use the familiar `<filename>{<spectrum_number>}` format to specify file names or you can pass a spectrum number as an argument. 

<img src="https://cdn.pixabay.com/photo/2012/11/28/11/16/star-67705_960_720.jpg" alt="drawing" width="400" align="center"/>


In the tutorial, there are some simulated Chandra data. Let's say that these data come from the observation of a a white drawf atmosphere. Let's see what we can do with these data.


In [ ]:
from threeML import *
# get xspec models
from astromodels.xspec import *

# this is the basic interface to 
# logging. see the documentation for more
update_logging_level("INFO")
silence_warnings()

import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u

%matplotlib notebook
from jupyterthemes import jtplot
jtplot.style(context='notebook', fscale=1, ticks=True, grid=False)


In [ ]:
chandra = OGIPLike(name="chandra", 
                   observation="c_data/obs.pha", 
                   background="c_data/obs_bak.pha",
                   response="c_data/acis.rmf",
                   arf_file="c_data/acis.arf",
                   spectrum_number=1 )

Note that 3ML probed the type of data that were read in. As long as the data files have been appropriately labelled, the plugin will **choose the correct likelihood for you**. While freedom is a great thing, math is not a democracy and thus we follow the rules so that your fits are of the highest scientific rigour. 

In this case, the total observation and the background observation are Poisson distributed. Thus, the proper likelihood is a Poisson for the total observation conditional on the Poisson likelihood of the background. For now, we will not model the background. Therefore a profile likelihood will be choosen.

Let's examine the properties of the plugin.


In [ ]:
chandra.significance

In [ ]:
chandra.significance_per_channel

In [ ]:
chandra.display_rsp()

In [ ]:
chandra.exposure

In [ ]:
chandra.view_count_spectrum(scale_background=True);

Now, not all channels are great to use in an analysis. Thus, we can set our selections.

In [ ]:
chandra.set_active_measurements?

In [ ]:
chandra.set_active_measurements('0.2-10')

In [ ]:
chandra.view_count_spectrum();

For profile likelihoods to be valid, there must be at least 1 [background count per channel](https://giacomov.github.io/Bias-in-profile-poisson-likelihood/). Let's do that here:

In [ ]:
chandra.rebin_on_background(1)
#chandra.remove_rebinning()

In [ ]:
chandra.view_count_spectrum();

## Fitting 

Ok, we are basically ready to do a fit. But we need a model. Let's make two models, one of a black body and the other a power law. We are going to be Bayesians for now, but remember, there is little difference between the interface for the two approaches.

### blackbody model


In [ ]:
bb = Blackbody()


#priors
bb.K.prior = Log_uniform_prior(lower_bound = 1e-2, upper_bound = 20)
bb.kT.prior = Truncated_gaussian(mu= 1, sigma=2, lower_bound=0, upper_bound=10)


# you can also try with the XSPEC blackbody!

# bb = XS_bbody()
# #priors
# bb.norm.prior = Log_uniform_prior(lower_bound = 1e-2, upper_bound = 20)
# bb.kt.prior = Truncated_gaussian(mu= 1, sigma=2, lower_bound=0, upper_bound=10)

# source
ps_bb = PointSource("white_drawf_bb", 0, 0, spectral_shape=bb)

# model
model_bb = Model(ps_bb)

In [ ]:
bayes_bb = BayesianAnalysis(model_bb, DataList(chandra))

# let's use ultranest this time
bayes_bb.set_sampler("ultranest")

bayes_bb.sampler.setup(min_num_live_points=400)

_ = bayes_bb.sample()

In [ ]:
bayes_bb.results.corner_plot();

In [ ]:
display_spectrum_model_counts(bayes_bb,
                              min_rate=50,
                              show_background=True);



In [ ]:
show_configuration('plugins')

In [ ]:
threeML_config.plugins.ogip.fit_plot.data_color = 'limegreen'
threeML_config.plugins.ogip.fit_plot.model_color = '#FF5AFD'

In [ ]:
display_spectrum_model_counts(bayes_bb,
                              min_rate=10,
                              step=True,
                              show_background=True);




### power law model

In [ ]:
plaw = Powerlaw()

plaw.K.prior = Log_uniform_prior(lower_bound = 1e-2, upper_bound = 10) 

plaw.index.prior = Gaussian(mu=-1, sigma=2)
plaw.index.bounds = (None, None)


# source
ps_pl = PointSource("white_drawf_pl", 0, 0, spectral_shape=plaw)

# model
model_pl = Model(ps_pl)

In [ ]:
bayes_pl = BayesianAnalysis(model_pl, DataList(chandra))

# let's use ultranest this time
bayes_pl.set_sampler("ultranest")

bayes_pl.sampler.setup(min_num_live_points=400)

_ = bayes_pl.sample()

In [ ]:
display_spectrum_model_counts(bayes_pl,
                              min_rate=10,
                              show_background=True);

In [ ]:
plot_spectra(bayes_bb.results, bayes_pl.results,
             flux_unit="erg/(cm2 s keV)",
             ene_min=1*u.keV, ene_max=10*u.keV);

## posterior predictive checks (PPC)

Let's use an external package to 3ML (but built with its tools box!) to compute posterior predictive checks. PPCs are model checking tool that integrate over the posterior and likelihood to compute the probability of new data from the observed data. We can compute this via simulating new data from the likelihood for sampled points from our posterior. A more detailed explanation for this can be found [here](https://academic.oup.com/mnras/article/490/1/927/5570608).

In [ ]:
from twopc import compute_ppc

In [ ]:
ppc_bb = compute_ppc(bayes_bb,
                  bayes_bb.results,
                  n_sims=500,
                  file_name="ppc_bb.h5",
                  return_ppc=True, overwrite=True)

In [ ]:
xx = ppc_bb.chandra.ppc_counts[0] - ppc_bb.chandra.scale_factor* ppc_bb.chandra.ppc_background[0]

In [ ]:
ppc_bb.chandra.plot(bkg_subtract=True);

In [ ]:
ppc_bb.chandra.plot_qq(channel_energies=None);

In [ ]:
ppc_pl = compute_ppc(bayes_pl,
                  bayes_pl.results,
                  n_sims=500,
                  file_name="ppc_pl.h5",
                  return_ppc=True, overwrite=True)

In [ ]:
ppc_pl.chandra.plot(bkg_subtract=True);

In [ ]:
ppc_pl.chandra.plot_qq(channel_energies=[1,2]);

## Background modeling

While profile likelihoods are a great tool when we have no good model for the background, if we can model the background, we should. It is possible to introduce models for the background in 3ML by extracting a plugin with the background spectrum only from a normal OGIPLike plugin. 


<img src="https://upload.wikimedia.org/wikipedia/commons/f/f1/Artist_impression_of_a_supermassive_black_hole_at_the_centre_of_a_galaxy.jpg" alt="drawing" width="400" align="center"/>

This time we look at some simulated data where we are trying to find a spectral line on top of a continuum. The source is a blackhole accreation disk. First, let's load the data and have a look




In [ ]:
chandra = OGIPLike(name="chandra", 
                   observation="c_data/obs_bkg_demo.pha", 
                   background="c_data/obs_bkg_demo_bak.pha",
                   response="c_data/acis.rmf",
                   arf_file="c_data/acis.arf",
                   spectrum_number=1 )


chandra.set_active_measurements("1-10")

chandra.rebin_on_background(1)

fig = chandra.view_count_spectrum(scale_background=True);

ax =fig.get_axes()[0]

ax.set_xlim(1,10)
ax.set_xscale('linear')





Now let's build our model and fit.

In [ ]:
cpl = Powerlaw(piv=5) + Gaussian()

#priors
cpl.K_1.prior = Log_uniform_prior(lower_bound = 1e-4, upper_bound = 1)
#cpl.xc_1.prior = Truncated_gaussian(mu= 5, sigma=3, lower_bound=1, upper_bound=12)
cpl.index_1.prior = Truncated_gaussian(mu= -2, sigma=1, lower_bound=-np.inf, upper_bound=0)

cpl.F_2.prior = Log_uniform_prior(lower_bound = 1e-3, upper_bound = 1)
cpl.mu_2.prior = Truncated_gaussian(mu= 6.4, sigma=2, lower_bound=1, upper_bound=10)
cpl.sigma_2.prior = Log_uniform_prior(lower_bound = 1e-3, upper_bound = 1)
# source
ps_blackhole = PointSource("blackhole", 0, 0, spectral_shape=cpl)

# model
model_blackhole = Model(ps_blackhole)





In [ ]:
bayes = BayesianAnalysis(model_blackhole, DataList(chandra))

# let's use ultranest this time
bayes.set_sampler("multinest")

bayes.sampler.setup(n_live_points=500,verbose=True                       )

In [ ]:
_ = bayes.sample()

In [ ]:
fig = chandra.display_model(step=False, min_rate=5,
                               data_color="#C39BD3", model_color="#76D7C4",
                               
                               )

ax = fig.get_axes()[0]

ax.set_xlim(1,10)

ax.set_xscale('linear')



In [ ]:
chandra.remove_rebinning()

bkg_plugin = SpectrumLike.from_background("bkg", chandra)

bkg_plugin.view_count_spectrum();


In [ ]:
bkg_spectrum = Powerlaw(K=1.e0,index=-1.2, piv=5) + Gaussian(F=1e0, mu=1, sigma=.2) 

bkg_spectrum.K_1.prior = Log_uniform_prior(lower_bound=1e-2, upper_bound=1e2)
bkg_spectrum.index_1.prior = Truncated_gaussian(mu = -1.2, sigma=0.5, lower_bound=-np.inf, upper_bound=0)

bkg_spectrum.F_2.prior = Log_uniform_prior(lower_bound=1e-2, upper_bound=1e2)
bkg_spectrum.mu_2.prior = Truncated_gaussian(mu = 3, sigma=1, lower_bound=0, upper_bound=10)
bkg_spectrum.sigma_2.fix = True


bkg_src = PointSource("bkg", 0,0, spectral_shape=bkg_spectrum)

bkg_model = Model(bkg_src)


bkg_plugin.set_model(bkg_model)



In [ ]:
chandra_bkg = OGIPLike("chandra",
                       observation = "c_data/obs_bkg_demo.pha",
                       response = "c_data/acis.rmf", 
                       arf_file = "c_data/acis.arf" ,
                       background = bkg_plugin,
                       spectrum_number=1
                        
                      
                      )


chandra_bkg.set_active_measurements("1-10")



In [ ]:
fig = chandra_bkg.view_count_spectrum();

ax =fig.get_axes()[0]

ax.set_xlim(1,10)
ax.set_xscale('linear')

In [ ]:
chandra_bkg.nuisance_parameters

In [ ]:


new_model = clone_model(model_blackhole)


bayes_bkg = BayesianAnalysis(new_model, DataList(chandra_bkg))


bayes_bkg.set_sampler("multinest")

bayes_bkg.sampler.setup(n_live_points=500,verbose=True )

In [ ]:
_ = bayes_bkg.sample()

In [ ]:
threeML_config.bayesian.corner_style.cmap = "magma"
threeML_config.bayesian.corner_style.extremes="none"

In [ ]:
bayes_bkg.results.corner_plot(color="#F6FF74");

In [ ]:
# over plot the joint background and source fits
fig = chandra_bkg.display_model(step=False, min_rate=1,
                               data_color="#C39BD3", model_color="#76D7C4",
                               
                               )

_ = bkg_plugin.display_model(
    data_color="#85C1E9", model_color="#F7DC6F",
    model_subplot=fig.axes,
    step=False,
    min_rate=1
)

ax = fig.get_axes()[0]

ax.set_xlim(1,10)

ax.set_xscale('linear')



The OGIP plugin (or any plugin) is not just for fitting, it can be used as a generic interface between models and isntruments for building pipelines. 
* Plugins and models are serializable meaning they can be farmed out to multi-processing
* Most plugins can simulate data from their likelihoods (complex instruments still need some work here)

Let's try this out: